# QAOA report

<img src="img/qaoa.png" width="55%" />

Picture copied from Blekos, K., Brand, D., Ceschini, A., Chou, C., Li, R., et. al. (2024). A review on Quantum Approximate Optimization Algorithm and its variants. Physics Reports, 1068, 1–66. https://doi.org/10.1016/j.physrep.2024.03.002

One layer of a QAOA circuit consists of a cost Hamiltonian and a mixer Hamiltonian. 

The cost Hamiltonian defines our problem and the mixer Hamiltonian can be thought as a function that allows us to explore different solutions to the problem. 

The variational parameters for the cost and mixer Hamiltonians are optimized using a classical computer. We optimize the parameters by minimizing the expectation value of the problem function.

In the standard QAOA the mixer Hamiltonian is $ \hat{H}_M = \sum_i^N \hat{X}_i $, where $N$ is the number of qubits.

## Variants

In the following three variants of QAOA, the circuit is initialized with a state $\ket{\psi_0} = \ket{+}^{\otimes N}$, where $N$ is the number of qubits. In all QAOA variants, adding more layers should produce more accurate solutions to the problem.

### Standard QAOA

### ma-QAOA

<img src="img/maqaoa.png?4" width="58%" />

Every gate in the cost and mixer layers are given a different variational parameter.

The mixer Hamiltonian is the same as in the standard QAOA.

Can give better results than the standard QAOA while using less layers.

The caveat is that there is more variational parameters when more layers are used, so the optimization takes longer.

Herrman, R., Lotshaw, P.C., Ostrowski, J. et al. Multi-angle quantum approximate optimization algorithm. Sci Rep 12, 6781 (2022). https://doi.org/10.1038/s41598-022-10555-8

### ADAPT-QAOA

ADAPT-QAOA can be problem-specific, since it can choose the best mixer Hamiltonian from a predefined pool for each layer of the QAOA. It should produce good results faster than the standard QAOA (fewer layers). <br>
ADAPT-QAOA should also reduce the amount of CNOT gates and variational parameters compared to the standard QAOA.

ADAPT-QAOA works in the following way:

Step 1: Define an initial parameter $\gamma_0$ and give it a small value, for example $\gamma_0 = 0.01$.

Step 2: Initialize the circuit with the state $\ket{\psi_p} = \ket{\psi_0} = \ket{+}^{\otimes N}$, where $N$ is the number of qubits. We denote the number of layers with $p$ and initally $p = 0$.

Step 3: Define an operator pool, which can include single-qubit mixers or both the single-qubit mixers and multi-qubit entangling gates. I have used the multi-qubit operator pool.

Step 4: Now we want to determine, which mixer we want to use for the layer $p+1$. Create a new state, where we apply the cost Hamiltonian with the initial parameter $\gamma_0$: $\;$ $\ket{\bar{\psi}_p} = \exp{(-i \gamma_0 \hat{H}_C)} \ket{\psi_p}$.

Step 5: Now we calculate the derivative of the cost Hamiltonian with respect to each operator in our pool. For the $i$ th operator, the derivative can be calculated as $ -i \braket{\bar{\psi}_p | \left[\hat{H}_C, \hat{A}_i \right] | \bar{\psi}_p}$.

(Step 6: Calculate the norm of the gradient. If it is below a predefined threshold, then we stop the algorithm.) Or run the ADAPT-QAOA with a predefined number of layers.

Step 7: Choose the mixer operator $\hat{A}_{max}$ for which the derivative was largest and use that in the layer $p+1$. The state at a layer $p+1$ is $\ket{\psi_{p+1}} = \exp{(-i \hat{A}_{max} \beta_{p+1})} \exp{(-i \hat{H}_C \gamma_{p+1})} \ket{\psi_p}$

Step 8: Optimize all the parameters in the circuit.

Step 9: Jump back to Step 3.

Zhu, L., Tang, H. L., Barron, G. S., Mayhall, N. J., Barnes, E., & Economou, S. E. (2022). Adaptive quantum approximate optimization algorithm for solving combinatorial problems on a quantum computer. Physical Review Research, 4(3). https://doi.org/10.1103/physrevresearch.4.033029

## Optimization routine

Shot-based expectation value calculation:
For example, when $10 000$ shots are used, we measure the circuit $10 000$ times and save the measured bitstrings.
Then we calculate the expectation value of every bitstring with respect to the QUBO matrix and calculate the average of those expectation values. This way we obtain the expectation value.

The total computational time of QAOA can get long when we are using $10 000$ shots and when we have many layers of QAOA (>20). In my testing, lower amount of shots than $10 000$ can affect the optimization routine negatively.

There are different optimization strategies (methods) that can be used. In the SciPy minimization function, it is easy to change the method. Examples of usable methods are COBYLA, Nelder-Mead and SLSQP. <br>
These methods use different strategies to minimize a function. In my testing, I have found that COBYLA works very well with all of the different QAOA variants. <br>
Nelder-Mead and SLSQP can take very long to complete and at least in my testing, they have given worse results than COBYLA.

## Initial parameters

A random number generator is used to pick a random initial parameter from $0$ to $2\pi$ for every beta and gamma. Sometimes these random initial parameters can be bad and running the optimization routine many times as explained above can help.

My guess is that when the initial parameters are bad, the optimization can get stuck in a local minimum although there would be a global minimum somewhere else.

## Converting from QUBO to Ising

Glover, F., Kochenberger, G., & Du, Y. (2018, November 13). A tutorial on formulating and using QUBO models. arXiv.org. https://arxiv.org/abs/1811.11538

Many optimization problems can be presented neatly by QUBO formulation. But when we are solving a problem at hand, we first need to convert it from the QUBO formulation to an Ising formulation. The Ising formulation gives us a Hamiltonian that we can use.

For example, consider a MaxCut problem given by the following graph:

![MaxCut](img/maxcut.png?1)

MaxCut can be represented in QUBO formulation where every node $i$ is represented by a polynomial term $x_i$ multiplied by the number of other nodes that the node $i$ is connected to and where every edge $(i,j)$ is represented by a quadratic term $x_i x_j$ multiplied by $2$.

MaxCut is usually represented by a maximizing function, but a maximizing function can be converted to a minimization function by the following formula:
$$ \min y = - \max y .$$

The QUBO formulation in a minimization form for the above graph is:
$$ \min y = -3x_0-2x_1-2x_2-3x_3-2x_4+2x_0x_1+2x_0x_3+2x_0x_4+2x_1x_2+2x_2x_3+2x_3x_4 .$$

Now this QUBO formulation can be converted to the Ising formulation by the following conversion:
$$ x_i \rightarrow \frac{1 - z_i}{2} .$$

Now the Ising formulation of this MaxCut problem becomes (after some calculations)
$$ \min y = \frac{1}{2}\left( z_0z_1 + z_0z_3 + z_0z_4 + z_1z_2 + z_2z_3 + z_3z_4 \right) - 3 .$$

For our optimization problem, we do not need to care about the offset number "$-3$". Now we can use this Ising formulation as a cost Hamiltonian by using Z operators in the place of every $z_i$:
$$ \hat{H}_C = \frac{1}{2}\left( \hat{Z}_0\hat{Z}_1 + \hat{Z}_0\hat{Z}_3 + \hat{Z}_0\hat{Z}_4 + \hat{Z}_1\hat{Z}_2 + \hat{Z}_2\hat{Z}_3 + \hat{Z}_3\hat{Z}_4 \right) .$$

The following function takes a QUBO matrix as an argument and returns an Ising Hamiltonian as a SparsePauliOp, which can be applied to a quantum circuit using Qiskit.

In [1]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp

def convert_qubo_to_ising(qubo):
    # Number of qubits
    n = len(qubo)

    # Calculate the offset also, this is not important for the optimization
    offset = 0
    
    operator_list = []

    for i in range(n):
        for j in range(i, n):
            # Initialize the Pauli operator with all I's
            pauli_operator = list("I"*n)
            
            if i == j:
                pauli_operator[i] = "Z"
                ising_value = -(1/2)*np.sum(qubo[i])
            else:
                pauli_operator[i] = "Z"
                pauli_operator[j] = "Z"
                ising_value = (1/2)*qubo[i][j]

            if not ising_value == 0:
                ising_pauli_op = (''.join(pauli_operator), ising_value)
                operator_list.append(ising_pauli_op)

            offset += (1/2)*qubo[i][j]

    operators = SparsePauliOp.from_list(operator_list)

    return operators

## Example QUBO problem by OP

The variables that can be changed in this problem are features, samples, alpha and seed. Features correspond to the amount of elements in the QUBO matrix. If we have $n$ features, then we have a $n \times n$ QUBO matrix. Alpha corresponds to "weighting the independence vs prediction power of variables". Changing alpha changes how diagonal and non-diagonal terms of the QUBO matrix correspond.

Example QUBO matrix generated with 5 features, 5 samples, 0.5 alpha and 51 seed:
$$Q = \begin{pmatrix}
-0.14765756 & 0.14765756 & 0.35215976 & 0.05033288 & 0.06933625\\
0.14765756 & -0.35215976 & 0.05218711 & 0.42657332 & 0.34000757\\
0.35215976 & 0.05218711 & -0.05033288 & 0.0313348 & 0.2343195\\
0.05033288 & 0.42657332 & 0.0313348 & -0.06933625 & 0.16140564\\
0.06933625 & 0.34000757 & 0.2343195 & 0.16140564 & -0.13338924
\end{pmatrix}$$

### Optimize in a normal way

The problem should become increasingly hard when more features are added (many different bitstrings result in a value that is close to the minimum value of the function).

In the plot below, the three variants of QAOA are compared to each other with a variable number of features. The ma-QAOA and ADAPT-QAOA are better when the number of features is less than $6$. When the number of features is more than $6$ ma-QAOA gets worse and APAPT-QAOA gets worse at number of features of $8$. This could suggest that the initial parameters were bad for the last cases of ma-QAOA and ADAPT-QAOA.

![Difference_Features](img/random_seed/variable_features/p5/features_difference.png?1)

The plot below shows a comparison between the QAOA variants with respect to a variable alpha. It seems that ma-QAOA is constantly better than the standard QAOA with a constant layer number of $5$. The only exception is at $\mathrm{alpha} = 0.9$. ADAPT-QAOA seems to be all over the place. This suggests that ADAPT-QAOA is problem-specific, since for some values of alpha it gives very good results and for some alpha it gives worse results. At $\mathrm{alpha} = 1.0$, the difference between the classical solver and ADAPT-QAOA is very close to $0$.

![Difference_Alpha](img/random_seed/variable_alpha/p5/alpha_difference.png?1)

The plot belows shows a comparison with an $\mathrm{alpha} = 0.2$ problem between the QAOA variants with respect to variable number of layers. We can see that ma-QAOA and ADAPT-QAOA perform almost constantly better than the standard QAOA.

$\mathrm{alpha} = 0.2$

![Difference_Layers_alpha0.2](img/random_seed/variable_layers/alpha0.2/layers_difference.png?1)

![time_plot_alpha0.2](img/random_seed/variable_layers/alpha0.2/time_plot.png?2)

### Optimization after every layer, first gamma and beta initially at interval $[0,2\pi]$ and after that gammas and betas initially $0$

From $\mathrm{alpha} = 0.2$ to $\mathrm{alpha} = 0.8$ with interval $0.2$.

![Difference_Layers_alpha0.2_newgammabetazero](img/beta_gamma_zero/variable_layers/alpha0.2/layers_difference.png?2)
![Difference_Layers_alpha0.4_newgammabetazero](img/beta_gamma_zero/variable_layers/alpha0.4/layers_difference.png?2)
![Difference_Layers_alpha0.6_newgammabetazero](img/beta_gamma_zero/variable_layers/alpha0.6/layers_difference.png?2)
![Difference_Layers_alpha0.8_newgammabetazero](img/beta_gamma_zero/variable_layers/alpha0.8/layers_difference.png?2)

### Optimization after every layer, every gamma and beta initially at interval $[0,2\pi]$.

From $\mathrm{alpha} = 0.2$ to $\mathrm{alpha} = 0.8$ with interval $0.2$.

![Difference_Layers_alpha0.2_opteverylayer](img/every_layer_optimization/variable_layers/alpha0.2/layers_difference.png?2)
![Difference_Layers_alpha0.4_opteverylayer](img/every_layer_optimization/variable_layers/alpha0.4/layers_difference.png?2)
![Difference_Layers_alpha0.6_opteverylayer](img/every_layer_optimization/variable_layers/alpha0.6/layers_difference.png?2)
![Difference_Layers_alpha0.8_opteverylayer](img/every_layer_optimization/variable_layers/alpha0.8/layers_difference.png?2)

Plot against different alphas, $5$ layers

![Difference_Alpha_5layers_opteverylayer](img/every_layer_optimization/variable_alpha/p5/alpha_difference.png?3)

### Convergence to value $|\langle E \rangle - E_{exact}| = 0.1$ and respective time plots

Alpha $0.2$

![Convergence_alpha0.2](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.2/layers_difference.png)
![Convergence_alpha0.2_timeplot](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.2/time_plot.png)

Alpha $0.4$

![Convergence_alpha0.4](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.4/layers_difference_2.png?2)
![Convergence_alpha0.4_timeplot](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.4/time_plot_2.png)

Alpha $0.6$

![Convergence_alpha0.6](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.6/layers_difference.png)
![Convergence_alpha0.6_timeplot](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.6/time_plot.png)

## Different seeds

Seeds: 51, 79, 196, 343, 514

### Alpha 0.2

![Convergence_alpha0.2_seeds](img/beta_gamma_zero/variable_layers/convergence/difference0.1/seeds/alpha0.2/layers_difference.png)

With alpha 0.2, on average it took standard QAOA 8 layers, ma-QAOA 17 layers and ADAPT-QAOA 13 layers to complete the algorithm.

### Alpha 0.4

For alpha 0.4 and alpha 0.6 the ma-QAOA did not want to complete in the time limit I set it to (20 hours).

![Convergence_alpha0.4_seeds](img/beta_gamma_zero/variable_layers/convergence/difference0.1/seeds/alpha0.4/layers_difference.png)

With alpha 0.4, on average it took standard QAOA 18 layers and ADAPT-QAOA 38 layers to complete the algorithm.

### Alpha 0.6

![Convergence_alpha0.6_seeds](img/beta_gamma_zero/variable_layers/convergence/difference0.1/seeds/alpha0.6/layers_difference.png)

With alpha 0.6, on average it took standard QAOA 48 layers and ADAPT-QAOA 45 layers to complete the algorithm.

It seems that with higher alpha, ADAPT-QAOA catches up to the standard QAOA.

## Different number of features

### Features = 5

![Convergence_alpha0.2_f5](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.2/features5_new.png)

With 5 features, on average it took standard QAOA 8 layers and ADAPT-QAOA 13 layers to complete the algorithm.

### Features = 6

![Convergence_alpha0.2_f6](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.2/features6_new.png)

With 6 features, on average it took standard QAOA 8 layers and ADAPT-QAOA 9 layers to complete the algorithm.

### Features = 7

![Convergence_alpha0.2_f7](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.2/features7_new.png)

With 7 features, on average it took standard QAOA 12 layers and ADAPT-QAOA 17 layers to complete the algorithm.

### Features = 8

![Convergence_alpha0.2_f8](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.2/features8_new.png?1)

With 8 features, on average it took standard QAOA 16 layers and ADAPT-QAOA 17 layers to complete the algorithm.

### Features = 9

![Convergence_alpha0.2_f9](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.2/features9_new.png)

With 9 features, on average it took standard QAOA 21 layers and ADAPT-QAOA 16 layers to complete the algorithm.

### Features = 10

![Convergence_alpha0.2_f10](img/beta_gamma_zero/variable_layers/convergence/difference0.1/alpha0.2/features10_new.png)

With 10 features, on average it took standard QAOA 18 layers and ADAPT-QAOA 20 layers to complete the algorithm.

## How to get WSL and Docker working (for example for using Qubernetes)

In order to use Docker in Windows, Windows Subsystem for Linux (WSL) needs to be installed.

WSL needs to be activated in a Windows system and the activation requires admin rights. For university laptops, send an email to ict@oulu.fi and ask them to activate it. In my case, this was not enough and I had to send a second message to ICT since WSL would not start. The error messages indicated that Virtual Machine feature needed to be enabled in Windows, which also requires admin rights. In order to check that WSL is working, you can do "wsl --status" in PowerShell. If no error messages are printed, WSL should be ready to go.

Next, install a Linux distribution for WSL from Microsoft Store. I installed Ubuntu distribution. After it has finished installing, it can be opened like a normal application in Windows or by typing "ubuntu" in PowerShell. Next, it asks for username and password, they do not need to match Windows user credentials. After that, you are done.

For Qubernetes, follow the "Using with Docker" instructions in https://www.qubernetes.dev/jupyter/getting-started

## Approximation ratio 0.878, optimization method changed to Powell

Seeds used: 51, 79, 196, 343 and 514

![Convergence_avg_logtime_qaoa](img/beta_gamma_zero/ratio/avgtime_logtime_qaoa.png?1)

With 12 features, the standard QAOA got stuck at approximation ratio 0.8 for the seed 343 and 0.7 for the seed 514 and the ratio did not get better in a reasonable time (over 15 hours). Now with the extrapolation it seems that for 20 features it would take around 1000 minutes (approx. 17 hours) to reach the desired accuracy, but in reality it would take longer since the 12 features case is not plotted.

![Convergence_avg_logtime_adapt](img/beta_gamma_zero/ratio/avgtime_logtime_adapt.png?1)

When the data is extrapolated to 20 features, it would take at least 10 000 minutes (approx. 7 days) to reach the desired accuracy.

Notes: connections between qubits increase as the problem size increases, warm-start, LR-QAOA, different optimization methods...